In [43]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go # 그래프 객체 생성
from plotly.subplots import make_subplots
import datetime
import seaborn as sns 
import scipy.stats
import plotly.express as px


In [44]:
### Data call 

## weather
weatherdata2017 = pd.read_csv("./ENITT_SOLAR_DATA/otherdata/서부발전_2017.csv", encoding="ms949")
weatherdata2018 = pd.read_csv("./ENITT_SOLAR_DATA/otherdata/서부발전_2018.csv", encoding="ms949")
weatherdata2019 = pd.read_csv("./ENITT_SOLAR_DATA/otherdata/서부발전_2019.csv", encoding="ms949")
weatherdata2020 = pd.read_csv("./ENITT_SOLAR_DATA/otherdata/서부발전_2020.csv", encoding="ms949")

weather_all = weatherdata2017.merge(weatherdata2018, how="outer").merge(weatherdata2019, how="outer").merge(weatherdata2020, how="outer")
weather_all_ansan = weather_all[weather_all["지점명"] == "수원"]
weather_all_ansan.info()

print(weatherdata2017)

## solar
othersolar = pd.read_csv("./ENITT_SOLAR_DATA/otherdata/otherdata.csv", encoding="euc-kr")
othersolar_ansan = othersolar[othersolar["발전기명"] == "안산연성정수장태양광"]

## make date
def make_date_list(start, end):
    date_index = pd.date_range(start=start, end=end, freq='H')
    
    return date_index

# date_list = make_date_list('2018-10-22 00:00', '2018-12-31 23:00')

## reset data
def make_data(data1, data2):
    new_data_list.append(data1) ## 24시 -> 00시
    for i in range(2, len(data2)-1):
        new_data_list.append(data2[i])  ## 01시 ~ 23시 -> 01시 ~ 23시

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30644 entries, 0 to 83197
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   지점         30644 non-null  int64  
 1   지점명        30644 non-null  object 
 2   일시         30644 non-null  object 
 3   기온(°C)     30643 non-null  float64
 4   강수량(mm)    2831 non-null   float64
 5   풍속(m/s)    30642 non-null  float64
 6   풍향(16방위)   30642 non-null  float64
 7   습도(%)      30641 non-null  float64
 8   일조(hr)     16777 non-null  float64
 9   일사(MJ/m2)  16775 non-null  float64
 10  전운량(10분위)  27582 non-null  float64
 11  운형(운형약어)   16391 non-null  object 
 12  지면온도(°C)   30589 non-null  float64
dtypes: float64(9), int64(1), object(3)
memory usage: 3.3+ MB
        지점 지점명                일시  기온(°C)  강수량(mm)  풍속(m/s)  풍향(16방위)  습도(%)  \
0      119  수원  2017-01-01 01:00     0.1      NaN      0.3         0     96   
1      119  수원  2017-01-01 02:00    -1.1      NaN      0.0         0   

In [45]:
## date index
date = make_date_list("2017-01-01", "2020-07-01") ## 2017. 01. 01 00:00 ~ 2020-06-30 
date_set = date[:-1] ## 2017. 01. 01 00:00 ~ 2020-06-30 23:00 

## data 
new_data_list = []
for i in range(len(othersolar_ansan)):
    data1 = 0.0 ## 00시 데이터는 모두 0임
    data2 = othersolar_ansan.iloc[i, :]
    make_data(data1, data2)

## confirm length

# print(len(date_set))
# print(len(new_data_list))

## make dataframe by datatimeindex
# solar df by datatimeindex
df_solar_ansan = pd.DataFrame(new_data_list, columns=["발전량"])
df_solar_ansan.index = date_set

df_solar_ansan = df_solar_ansan[df_solar_ansan.index != datetime.datetime(2017, 1, 1, 0 , 0 ,0)]
df_solar_ansan = df_solar_ansan[df_solar_ansan.index != datetime.datetime(2018, 1, 1, 0 , 0 ,0)]
df_solar_ansan = df_solar_ansan[df_solar_ansan.index != datetime.datetime(2019, 1, 1, 0 , 0 ,0)]
df_solar_ansan = df_solar_ansan[df_solar_ansan.index != datetime.datetime(2020, 1, 1, 0 , 0 ,0)]


# date set => 01 01 00:00 data 삭제하기 (weather에 01 01 00 00 정보가 없음)
dummy_date = date_set
date_reset = dummy_date[dummy_date != datetime.datetime(2017, 1, 1, 0 , 0 ,0)]
date_reset = date_reset[date_reset != datetime.datetime(2018, 1, 1, 0 , 0 ,0)]
date_reset = date_reset[date_reset != datetime.datetime(2019, 1, 1, 0 , 0 ,0)]
date_reset = date_reset[date_reset != datetime.datetime(2020, 1, 1, 0 , 0 ,0)]

# weather df by datatimeindex

weather_all_ansan.index = date_reset
df_weather_ansan = weather_all_ansan
# original columns '지점', '지점명', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)','일조(hr)', '일사(MJ/m2)', '전운량(10분위)', '운형(운형약어)', '지면온도(°C)']
df_weather_ansan.columns = ['지점', '지점명', '일시', '기온', '강수량', '풍속', '풍향', '습도', '일조', '일사', '전운량', '운형', '지면온도']




In [46]:
### Data mean and sum
## mean -> 기온, 풍속, 습도, 지면온도, 전운량
## sum -> 일조, 일사, 전운량, 강수량
## delete -> 지점, 풍향

dummy_weather_formean = df_weather_ansan[['기온', '풍속', '습도', '지면온도', '전운량']]
dummy_weather_forsum = df_weather_ansan[['일조', '일사', '전운량', '강수량']]

## resample sum
def resample_date_sum(x): ## D = Day, T = second, min = minute, W = weekend?, M = months, Q = quarters, Y= Year
    return dummy_weather_forsum.resample(x).sum(), df_solar_ansan.resample(x).sum()

## resample mean
def resample_date_mean(x):
    return dummy_weather_formean.resample(x).mean(), df_solar_ansan.resample(x).mean()

## combine sum, mean 
# def combine():
#     resample_date_mean[0], resample_date_sum[0]

## resample Day
df_weather_forsum_D, df_solar_forsumD = resample_date_sum('D')
df_weather_formean_D, df_solar_formean = resample_date_mean('D')
memory_datatimeindexD = df_weather_forsum_D.index


df_weather_resampleD = df_weather_forsum_D.merge(df_weather_formean_D, on=df_weather_forsum_D.index, how="inner")
df_weather_resampleD.index = memory_datatimeindexD
df_weather_resample_D = df_weather_resampleD.drop(["key_0"], axis=1)

df_weather_resample_D # 전운량_x = sum, 전운량_y = mean

## resample months
df_weather_forsum_m, df_solar_forsumM = resample_date_sum('M')
df_weather_formean_m, df_solar_formean = resample_date_mean('M')
memory_datatimeindexM = df_weather_forsum_m.index


df_weather_resampleM = df_weather_forsum_m.merge(df_weather_formean_m, on=df_weather_forsum_m.index, how="inner")
df_weather_resampleM.index = memory_datatimeindexM
df_weather_resample_M = df_weather_resampleM.drop(["key_0"], axis=1)

df_weather_resample_M # 전운량_x = sum, 전운량_y = mean


## resample weekend
df_weather_forsum_W, df_solar_forsumW = resample_date_sum('W')
df_weather_formean_W, df_solar_formean = resample_date_mean('W')
memory_datatimeindexW = df_weather_forsum_W.index


df_weather_resampleW = df_weather_forsum_W.merge(df_weather_formean_W, on=df_weather_forsum_W.index, how="inner")
df_weather_resampleW.index = memory_datatimeindexW
df_weather_resample_W = df_weather_resampleW.drop(["key_0"], axis=1)

df_weather_resample_W # 전운량_x = sum, 전운량_y = mean

## resample months
df_weather_forsum_m, df_solar_forsumM = resample_date_sum('M')
df_weather_formean_m, df_solar_formean = resample_date_mean('M')
memory_datatimeindexM = df_weather_forsum_m.index


df_weather_resampleM = df_weather_forsum_m.merge(df_weather_formean_m, on=df_weather_forsum_m.index, how="inner")
df_weather_resampleM.index = memory_datatimeindexM
df_weather_resample_M = df_weather_resampleM.drop(["key_0"], axis=1)

df_weather_resample_M # 전운량_x = sum, 전운량_y = mean


## resample Q
df_weather_forsum_Q, df_solar_forsumQ = resample_date_sum('Q')
df_weather_formean_Q, df_solar_formean = resample_date_mean('Q')
memory_datatimeindexQ = df_weather_forsum_Q.index


df_weather_resampleQ = df_weather_forsum_Q.merge(df_weather_formean_Q, on=df_weather_forsum_Q.index, how="inner")
df_weather_resampleQ.index = memory_datatimeindexQ
df_weather_resample_Q = df_weather_resampleQ.drop(["key_0"], axis=1)

df_weather_resample_Q # 전운량_x = sum, 전운량_y = mean

## resample Y
df_weather_forsum_Y, df_solar_forsumY = resample_date_sum('Y')
df_weather_formean_Y, df_solar_formean = resample_date_mean('Y')
memory_datatimeindexY = df_weather_forsum_Y.index


df_weather_resampleY = df_weather_forsum_Y.merge(df_weather_formean_Y, on=df_weather_forsum_Y.index, how="inner")
df_weather_resampleY.index = memory_datatimeindexY
df_weather_resample_Y = df_weather_resampleY.drop(["key_0"], axis=1)

df_weather_resample_Y # 전운량_x = sum, 전운량_y = mean

## SEE DATA
# df_weather_resample_D, df_solar_forsumD
# df_weather_resample_W , df_solar_forsumW
# df_weather_resample_M , df_solar_forsumM
# df_weather_resample_Q , df_solar_forsumQ
print(len(df_weather_resample_Y), len(df_solar_forsumY)) 

4 4


In [47]:
## show graph D
## df_weather_resample_D, df_solar_forsumD


def showgraphD(x):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=df_solar_forsumD.index, y=df_solar_forsumD["발전량"], mode='lines', name="발전량", line=dict(color='blue')), secondary_y = False)
    fig.add_trace(go.Scatter(x=df_weather_resample_D.index, y=df_weather_resample_D[x], mode='lines', name=x, line=dict(color='red')), secondary_y = True)
    fig.update_layout(width=1000, height=300,)
    fig.show()
#  일조       일사    전운량_x     강수량         기온        풍속         습도  지면온도     전운량_y  
for i in df_weather_resample_D.columns:
    showgraphD(i)

## show scipy.stats

import scipy.stats
def pearsonrD(y):
    print(y, ": ", scipy.stats.pearsonr(df_solar_forsumD["발전량"], df_weather_resample_D[y]))
    return scipy.stats.pearsonr(df_solar_forsumD["발전량"], df_weather_resample_D[y])

all_pearsonrD = []
for i in df_weather_resample_D.columns:
    all_pearsonrD.append(pearsonrD(i))

pearsonD_df = pd.DataFrame(all_pearsonrD, columns=["상관계수", "p-value"])
pearsonD_df.index = df_weather_resample_D.columns

fig = go.Figure(data=[
    go.Bar(name='상관계수', x=pearsonD_df.index, y=pearsonD_df["상관계수"]),
    go.Bar(name='value', x=pearsonD_df.index, y=pearsonD_df["p-value"])
])
fig.show()


일조 :  (0.8978164324564213, 0.0)
일사 :  (0.8840646729350956, 0.0)
전운량_x :  (-0.6922938798336591, 7.055672228317597e-183)
강수량 :  (-0.34726110940855404, 1.6901597020806258e-37)
기온 :  (0.19060918990325867, 6.5184160111862346e-12)
풍속 :  (0.11042487001769857, 7.676492195634521e-05)
습도 :  (-0.4957708647960396, 3.585689737694931e-80)
지면온도 :  (0.26497725620127427, 5.834135938368805e-22)
전운량_y :  (-0.6863647605028109, 1.4608335906529063e-178)


In [48]:
## show graph W
## df_weather_resample_W, df_solar_forsumW


def showgraphW(x):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=df_solar_forsumW.index, y=df_solar_forsumW["발전량"], mode='lines', name="발전량", line=dict(color='blue')), secondary_y = False)
    fig.add_trace(go.Scatter(x=df_weather_resample_W.index, y=df_weather_resample_W[x], mode='lines', name=x, line=dict(color='red')), secondary_y = True)
    fig.update_layout(width=1000, height=300,)
    fig.show()
#  일조       일사    전운량_x     강수량         기온        풍속         습도  지면온도     전운량_y  
for i in df_weather_resample_W.columns:
    showgraphW(i)

## show scipy.stats

import scipy.stats
def pearsonrW(y):
    print(y, ": ", scipy.stats.pearsonr(df_solar_forsumW["발전량"], df_weather_resample_W[y]))
    return scipy.stats.pearsonr(df_solar_forsumW["발전량"], df_weather_resample_W[y])

all_pearsonrW = []
for i in df_weather_resample_W.columns:
    all_pearsonrW.append(pearsonrW(i))

pearsonW_df = pd.DataFrame(all_pearsonrW, columns=["상관계수", "p-value"])
pearsonW_df.index = df_weather_resample_W.columns

fig = go.Figure(data=[
    go.Bar(name='상관계수', x=pearsonW_df.index, y=pearsonW_df["상관계수"]),
    go.Bar(name='value', x=pearsonW_df.index, y=pearsonW_df["p-value"])
])
fig.show()


일조 :  (0.8574846433348571, 2.045706339011248e-54)
일사 :  (0.8676695808847753, 3.904071039709144e-57)
전운량_x :  (-0.3559555283702541, 7.089382508623331e-07)
강수량 :  (-0.2832632582109307, 9.767695032143392e-05)
기온 :  (0.3368768503244723, 2.926913511464762e-06)
풍속 :  (0.28698669794897036, 7.823041934571916e-05)
습도 :  (-0.41118881099158744, 6.705251860287942e-09)
지면온도 :  (0.399158800523138, 1.993049748866e-08)
전운량_y :  (-0.4448116176853135, 2.5172710172125493e-10)


In [49]:
## show graph M
# df_weather_resample_M , df_solar_forsumM
def showgraphM(x):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=df_solar_forsumM.index, y=df_solar_forsumM["발전량"], mode='lines', name="발전량", line=dict(color='blue')), secondary_y = False)
    fig.add_trace(go.Scatter(x=df_weather_resample_M.index, y=df_weather_resample_M[x], mode='lines', name=x, line=dict(color='red')), secondary_y = True)
    fig.update_layout(width=1000, height=300,)
    fig.show()
#  일조       일사    전운량_x     강수량         기온        풍속         습도  지면온도     전운량_y  
for i in df_weather_resample_M.columns:
    showgraphM(i)

## show scipy.stats

import scipy.stats
def pearsonrM(y):
    return scipy.stats.pearsonr(df_solar_forsumM["발전량"], df_weather_resample_M[y])

all_pearsonrM = []
for i in df_weather_resample_M.columns:
    # print(pearsonrM(i))
    all_pearsonrM.append(pearsonrM(i))

pearsonM_df = pd.DataFrame(all_pearsonrM, columns=["상관계수", "p-value"])
pearsonM_df.index = df_weather_resample_M.columns

fig = go.Figure(data=[
    go.Bar(name='상관계수', x=pearsonM_df.index, y=pearsonM_df["상관계수"]),
    go.Bar(name='value', x=pearsonM_df.index, y=pearsonM_df["p-value"])
])
fig.show()


In [50]:
## show graph Q
# df_weather_resample_Q , df_solar_forsumQ
def showgraphQ(x):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=df_solar_forsumQ.index, y=df_solar_forsumQ["발전량"], mode='lines', name="발전량", line=dict(color='blue')), secondary_y = False)
    fig.add_trace(go.Scatter(x=df_weather_resample_Q.index, y=df_weather_resample_Q[x], mode='lines', name=x, line=dict(color='red')), secondary_y = True)
    fig.update_layout(width=1000, height=300,)
    fig.show()
#  일조       일사    전운량_x     강수량         기온        풍속         습도  지면온도     전운량_y  
for i in df_weather_resample_Q.columns:
    showgraphQ(i)

## show scipy.stats

import scipy.stats
def pearsonrQ(y):
    return scipy.stats.pearsonr(df_solar_forsumQ["발전량"], df_weather_resample_Q[y])

all_pearsonrQ = []
for i in df_weather_resample_Q.columns:
    # print(pearsonrM(i))
    all_pearsonrQ.append(pearsonrQ(i))

pearsonQ_df = pd.DataFrame(all_pearsonrQ, columns=["상관계수", "p-value"])
pearsonQ_df.index = df_weather_resample_Q.columns

fig = go.Figure(data=[
    go.Bar(name='상관계수', x=pearsonQ_df.index, y=pearsonQ_df["상관계수"]),
    go.Bar(name='value', x=pearsonQ_df.index, y=pearsonQ_df["p-value"])
])
fig.show()


In [51]:
## show graph Y
# df_weather_resample_Y , df_solar_forsumY
def showgraphY(x):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=df_solar_forsumY.index, y=df_solar_forsumY["발전량"], mode='lines', name="발전량", line=dict(color='blue')), secondary_y = False)
    fig.add_trace(go.Scatter(x=df_weather_resample_Y.index, y=df_weather_resample_Y[x], mode='lines', name=x, line=dict(color='red')), secondary_y = True)
    fig.update_layout(width=1000, height=300,)
    fig.show()
#  일조       일사    전운량_x     강수량         기온        풍속         습도  지면온도     전운량_y  
for i in df_weather_resample_Y.columns:
    showgraphY(i)

## show scipy.stats

import scipy.stats
def pearsonrY(y):

    return scipy.stats.pearsonr(df_solar_forsumY["발전량"], df_weather_resample_Y[y])

all_pearsonrY = []
for i in df_weather_resample_Y.columns:
    # print(pearsonrM(i))
    all_pearsonrY.append(pearsonrY(i))

pearsonY_df = pd.DataFrame(all_pearsonrY, columns=["상관계수", "p-value"])
pearsonY_df.index = df_weather_resample_Y.columns

fig = go.Figure(data=[
    go.Bar(name='상관계수', x=pearsonY_df.index, y=pearsonY_df["상관계수"]),
    go.Bar(name='value', x=pearsonY_df.index, y=pearsonY_df["p-value"])
])
fig.show()


추가할 것 

1. 시간별 slice 하기 ( 06~ 19 )
2. 4년치 같은 날자로 묶기 -> mean으로 묶기 
2-1. 달별로 슬라이스 해서 확인하기, 1월달 1~30일까지 (시간별, 날자별 데이터 확인) 
2-2. 사계절별 (3~5, 6~8, 9~11, 12~2) 별로 묶고 각 시간별 날자별 데이터 확인 
 
상관계수, p-value값 정리


In [52]:
# df_weather_resample_D, df_solar_forsumD
# df_weather_resample_W , df_solar_forsumW
# df_weather_resample_M , df_solar_forsumM
# df_weather_resample_Q , df_solar_forsumQ
# print(datetime.date(2018, 1, 1))
# print(df_weather_resample_D.iloc[700: , :].head(50))
# df_weather_resample_D[df_weather_resample_D.index == datetime.date(2018, 1, 1)] 

## 2017 2.1	4.55	104.0	0.0	2.830435 0.691304	85.304348	-0.160870	6.117647
## 2019 5.3   6.93   59.0   0.0  -4.195652  1.891304  64.565217 -4.169565 2.565217 

In [53]:
# truncated to Months
# series_ts.truncate(before='2019-12-01')

def truncated(year):

    test_dummy = []
    for i in range(1, 13):
        if i == 2:
            j = 28
            test_dummy.append(df_weather_resample_D.truncate(after=f'{year}-{i}-{j}', before=f'{year}-{i}-01'))

        elif i == 1 or i == 3 or i == 5 or i == 7 or i == 8 or i == 10 or i == 12:
            j = 31
            test_dummy.append(df_weather_resample_D.truncate(after=f'{year}-{i}-{j}', before=f'{year}-{i}-01'))
        else:
            j = 30
            test_dummy.append(df_weather_resample_D.truncate(after=f'{year}-{i}-{j}', before=f'{year}-{i}-01')) 

    return test_dummy

dummy_dateM_list1 = [x for x in range(1, 32)]
dummy_dateM_list2 = [x for x in range(1, 31)]
dummy_dateM_list3 = [x for x in range(1, 29)]
# print(dummy_date_list1)
# def merge():

# weatherdata2017.merge(weatherdata2018, how="outer").merge(weatherdata2019, how="outer").merge(weatherdata2020, how="outer")

# test2017.merge(test2018, how="inner")

test2017M = truncated('2017')
test2018M = truncated('2018')
test2019M = truncated('2019')
# test2020 = truncated('2020')
# df_weather_resample_D.truncate(after='2017-01-31')
# test2018["year"] = test2018.index
for i in range(1, 13):
    dummy_date_list = []
    if i == 2:
        dummy_date_list = dummy_dateM_list3
        test2017M[i-1].index = dummy_date_list
        test2018M[i-1].index = dummy_date_list
        test2019M[i-1].index = dummy_date_list
        # test2020[i-1].index = dummy_date_list

    elif i == 1 or i == 3 or i == 5 or i == 7 or i == 8 or i == 10 or i == 12:
        dummy_date_list = dummy_dateM_list1
        test2017M[i-1].index = dummy_date_list
        test2018M[i-1].index = dummy_date_list
        test2019M[i-1].index = dummy_date_list
        # test2020[i-1].index = dummy_date_list
    else:
        dummy_date_list = dummy_dateM_list2
        test2017M[i-1].index = dummy_date_list
        test2018M[i-1].index = dummy_date_list
        test2019M[i-1].index = dummy_date_list
        # test2020[i-1].index = dummy_date_list

def sum_meanM(a, b, c):
    return (a + b + c) / 3

resultM = []
for i in range(12):
    resultM.append(sum_meanM(test2017M[i], test2018M[i], test2019M[i]))


resultM[0]
## resultM 0~11 : 1월 ~ 12d월

,일조,일사,전운량_x,강수량,기온,풍속,습도,지면온도,전운량_y
1,5.233333,6.346667,64.000000,0.000000,-1.040580,1.198551,68.057971,-1.821739,3.462916
2,6.100000,6.940000,60.000000,0.200000,-0.487500,1.465278,66.888889,-1.594444,3.324074
3,8.900000,8.860000,16.000000,0.000000,-1.712500,1.468056,56.750000,-2.227778,0.879630
4,5.700000,6.573333,84.000000,0.000000,-1.022222,0.990278,65.097222,-1.411111,4.194444
5,5.300000,7.163333,85.333333,0.000000,-0.487500,1.711111,59.486111,-1.550000,4.587963
6,8.066667,8.563333,53.333333,0.000000,-0.026389,1.559722,58.222222,-1.269444,2.625000
7,5.466667,6.743333,84.333333,0.000000,-0.431944,0.776389,71.652778,-1.315278,4.319444
8,4.066667,6.276667,116.666667,0.666667,1.916667,1.984722,57.611111,-0.073611,5.732038
9,7.366667,7.466667,109.666667,0.266667,-1.493056,2.213889,56.916667,-0.830556,5.077087
10,6.700000,7.070000,82.000000,0.333333,-3.086111,2.231944,55.597222,-1.304167,3.693519


In [54]:
### truncated to season
def truncated(year):
    yearplus = str(int(year) + 1)
    test_dummy = []
    for i in range(4):
        if i == 0:
            test_dummy.append(df_weather_resample_D.truncate(after=f'{year}-5-31', before=f'{year}-03-01'))
        elif i == 1:
            test_dummy.append(df_weather_resample_D.truncate(after=f'{year}-08-31', before=f'{year}-06-01'))
        elif i == 2:
            test_dummy.append(df_weather_resample_D.truncate(after=f'{year}-11-30', before=f'{year}-09-01')) 
        elif i == 3:
            test_dummy.append(df_weather_resample_D.truncate(after=f'{yearplus}-02-28', before=f'{year}-12-01')) 

    return test_dummy

dummy_num1 = pd.date_range(start="2017-03-01", end="2017-05-31")
dummy_num2 = pd.date_range(start="2017-06-01", end="2017-08-31")
dummy_num3 = pd.date_range(start="2017-09-01", end="2017-11-30")
dummy_num4 = pd.date_range(start="2017-12-01", end="2018-02-28")

dummy_dateS_list1 = [x for x in range(1, len(dummy_num1)+1)]
dummy_dateS_list2 = [x for x in range(1, len(dummy_num2)+1)]
dummy_dateS_list3 = [x for x in range(1, len(dummy_num3)+1)]
dummy_dateS_list4 = [x for x in range(1, len(dummy_num4)+1)]

# # weatherdata2017.merge(weatherdata2018, how="outer").merge(weatherdata2019, how="outer").merge(weatherdata2020, how="outer")

# # test2017.merge(test2018, how="inner")

test2017S = truncated('2017')
test2018S = truncated('2018')
test2019S = truncated('2019')
# # test2020 = truncated('2020')
# # df_weather_resample_D.truncate(after='2017-01-31')
# # test2018["year"] = test2018.index
for i in range(4):
    dummy_date_list = []
    if i == 0:
        dummy_date_list = dummy_dateS_list1
        test2017S[i].index = dummy_date_list
        test2018S[i].index = dummy_date_list
        test2019S[i].index = dummy_date_list
        # test2020[i-1].index = dummy_date_list

    elif i == 1:
        dummy_date_list = dummy_dateS_list2
        test2017S[i].index = dummy_date_list
        test2018S[i].index = dummy_date_list
        test2019S[i].index = dummy_date_list
#         # test2020[i-1].index = dummy_date_list
    elif i == 2:
        dummy_date_list = dummy_dateS_list3
        test2017S[i].index = dummy_date_list
        test2018S[i].index = dummy_date_list
        test2019S[i].index = dummy_date_list
#         # test2020[i-1].index = dummy_date_list
    else:
        dummy_date_list = dummy_dateS_list4
        test2017S[i].index = dummy_date_list
        test2018S[i].index = dummy_date_list
        test2019S[i].index = dummy_date_list

def sum_meanS(a, b, c):
    return (a + b + c) / 3

resultS = []
for i in range(4):
    resultS.append(sum_meanS(test2017S[i], test2018S[i], test2019S[i]))


resultS[0]
## resultS 0~3 : 3~5월, 6~8월, 9월~11월, 12월~2월

,일조,일사,전운량_x,강수량,기온,풍속,습도,지면온도,전운량_y
1,6.666667,11.866667,105.666667,2.600000,3.454167,2.383333,67.986111,3.583333,5.058745
2,8.600000,14.153333,86.666667,0.766667,2.529167,1.855556,58.208333,3.119444,3.979630
3,9.633333,13.876667,79.666667,0.000000,6.008333,1.337500,52.513889,4.194444,3.734893
4,6.466667,10.883333,102.000000,1.233333,7.668056,1.322222,74.222222,6.022222,4.662698
5,5.666667,11.183333,150.333333,5.533333,5.973611,2.120833,75.708333,5.775000,6.853175
...,...,...,...,...,...,...,...,...,...
88,4.966667,13.926667,115.333333,2.666667,18.401389,2.161111,68.083333,20.729167,5.857143
89,11.566667,24.523333,40.000000,0.000000,19.530556,2.016667,57.000000,22.279167,2.016835
90,9.166667,21.363333,69.333333,2.866667,20.580556,1.820833,68.750000,23.856944,3.163721
91,8.666667,18.846667,133.333333,2.000000,20.277778,1.965278,67.875000,22.902778,6.415741


In [14]:
def testGraph(x):
    fig = make_subplots()

    fig.add_trace(go.Scatter(x=resultS[0].index, y=resultS[0][x], mode='lines', name="봄", line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=resultS[1].index, y=resultS[1][x], mode='lines', name="여름", line=dict(color='red')))
    fig.add_trace(go.Scatter(x=resultS[2].index, y=resultS[2][x], mode='lines', name="가을", line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=resultS[3].index, y=resultS[3][x], mode='lines', name="겨울", line=dict(color='green')))
    fig.update_layout(width=1000, height=700)
    fig.show()

testGraph("일사")


In [15]:
resultS[0].corr()

,일조,일사,전운량_x,강수량,기온,풍속,습도,지면온도,전운량_y
일조,1.000000,0.901840,-0.843082,-0.449042,0.296653,-0.166865,-0.493906,0.351262,-0.839545
일사,0.901840,1.000000,-0.674039,-0.372448,0.615104,-0.142002,-0.383159,0.663139,-0.669936
전운량_x,-0.843082,-0.674039,1.000000,0.499434,-0.031074,0.189845,0.445167,-0.071574,0.986662
강수량,-0.449042,-0.372448,0.499434,1.000000,0.129742,0.148208,0.438194,0.105162,0.442262
기온,0.296653,0.615104,-0.031074,0.129742,1.000000,-0.112718,0.109737,0.989346,-0.041524
풍속,-0.166865,-0.142002,0.189845,0.148208,-0.112718,1.000000,-0.092087,-0.089285,0.190468
습도,-0.493906,-0.383159,0.445167,0.438194,0.109737,-0.092087,1.000000,0.114739,0.433112
지면온도,0.351262,0.663139,-0.071574,0.105162,0.989346,-0.089285,0.114739,1.000000,-0.079114
전운량_y,-0.839545,-0.669936,0.986662,0.442262,-0.041524,0.190468,0.433112,-0.079114,1.000000


In [88]:

df_weather_rain = df_weather_ansan.fillna(0)

df_weather_rain_on = df_weather_rain[df_weather_rain["강수량"] != 0]

df_weather_forsum_D_rain_on = df_weather_forsum_D[df_weather_forsum_D["강수량"] != 0]
for i in df_weather_forsum_D_rain_on.index:
    j = 0
    if i in df_solar_forsumD.index:
        j = j + 1
    else 
        df_solar_forsumD

df_solar_rain_on = df_solar_forsumD.loc[(df_weather_rain_on.index == df_solar_ansan.inde)]
# df_weather_forsum_D
# df_solar_ansan.loc[df_weather_rain_on.index == df_solar_ansan.index]
# df_weather_rain

,일조,일사,전운량,강수량
2017-01-02,3.3,4.94,137.0,0.6
2017-01-09,6.0,6.45,104.0,0.2
2017-01-12,7.4,6.04,110.0,1.9
2017-01-13,5.9,6.57,64.0,0.4
2017-01-20,6.5,6.96,99.0,3.9
...,...,...,...,...
2020-06-24,0.0,2.72,238.0,13.8
2020-06-25,0.0,6.78,240.0,4.3
2020-06-26,7.5,16.96,190.0,2.5
2020-06-29,2.2,8.35,196.0,20.0


추가로 할 것 ,, 
slice를 해야한다. 어떻게??
달별로 slice해서

1. 시간별로 slice -> 06~ 19 까지 해서 비교하기 ,, 
2. 4년 치를 같은 일자는 묶기, mean으로 묶는다. -> 상관계수와 p-value 값의 변화를 확인하자. </br>
2-1. ex) 달별 1월달 1월 1~30일까지 데이터만 가져와서 날자랑 비교하기 ( 달별 하루하루 데이터, 달별 시간당 데이터) </br>
2-2. ex) 사계절별(3~5, 6~8, 9~11, 11~2) 데이터만 가져와서 날짜랑 비교하기 

=> 달별 상관계수와 p-value값 정리
=> 사계절별 상관계수와 p-value값 정리